# Food Image Classification with Tensorflow

## Load Food Images Dataset
First, we need to download Food Images Dataset from google drive.


In [1]:
!gdown 1ViO8VfaZ_MpvGVFnRzVrbowSVP2D9Ys0

Downloading...
From: https://drive.google.com/uc?id=1ViO8VfaZ_MpvGVFnRzVrbowSVP2D9Ys0
To: /content/food.zip
100% 3.73G/3.73G [00:26<00:00, 140MB/s] 


In [2]:
import zipfile
import os
import shutil
import random

# Unzip the dataset
local_zip = './food.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./images')
zip_ref.close()

In [3]:
!rm food.zip

In [4]:
foods = os.listdir('images')

## Split Dataset into Train, Valid, and Test

In [5]:
# Assign training and validation set directories
base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Directory with training pictures
train_dirs = [os.path.join(train_dir, str(food)) for food in foods]

# Directory with validation pictures
valid_dirs = [os.path.join(valid_dir, str(food)) for food in foods]

# Directory with test pictures
test_dirs = [os.path.join(test_dir, str(food)) for food in foods]

In [6]:
for dir in train_dirs+valid_dirs+test_dirs:
    try:
        os.makedirs(dir)
    except:
        pass

In [7]:
def split_data(source, train, valid, test, split=[80,15,5]):
    """Split dataset into train, valid, and test."""
    files = os.listdir(source)
    files = random.sample(files, len(files))
    train_num = int((split[0]/sum(split))*len(files))
    valid_num = int((split[1]/sum(split))*len(files))
    for file in files:
        if len(os.listdir(train)) < train_num:
            shutil.copy(os.path.join(source, file), train)
        elif len(os.listdir(valid)) < valid_num:
            shutil.copy(os.path.join(source, file), valid)
        else:
            shutil.copy(os.path.join(source, file), test)

In [8]:
for i, food in enumerate(foods):
    split_data(f'images/{food}/', train_dirs[i], valid_dirs[i], test_dirs[i])

In [9]:
train_path = 'data/train'
valid_path = 'data/valid'
test_path = 'data/test'

## Build Transfer Learning Model
We use Mobilenet V2 as base model, we only use part of its layers, and add Dense layer on the bottom of that.

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

Build dataset to feed the model.

In [17]:
train_datagen = ImageDataGenerator(
                  preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).\
                  flow_from_directory(
                  directory=train_path, target_size=(224,224), batch_size=10)

valid_datagen = ImageDataGenerator(
                  preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).\
                  flow_from_directory(
                  directory=valid_path, target_size=(224,224), batch_size=10)

test_datagen = ImageDataGenerator(
                  preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).\
                  flow_from_directory(
                  directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 7838 images belonging to 49 classes.
Found 1469 images belonging to 49 classes.
Found 490 images belonging to 49 classes.


In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [ ]:
# mobile.summary()

In [ ]:
x = mobile.layers[-5].output
x = layers.Reshape((1024,))(x)
output = layers.Dense(49, activation='softmax')(x)

In [ ]:
model = Model(inputs=mobile.input, outputs=output)

In [ ]:
for layer in model.layers[:-23]:
  layer.trainable = False

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_datagen,
             validation_data=valid_datagen,
             epochs=30,
             verbose=1)

Epoch 1/30
 11/785 [..............................] - ETA: 3:39 - loss: 4.6920 - accuracy: 0.0364

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


 89/785 [==>...........................] - ETA: 3:52 - loss: 3.7762 - accuracy: 0.1067

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


427/785 [===============>..............] - ETA: 2:01 - loss: 2.5440 - accuracy: 0.3731

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag)


785/785 [==============================] - 311s 394ms/step - loss: 2.0897 - accuracy: 0.4732 - val_loss: 1.2475 - val_accuracy: 0.6610
Epoch 2/30
785/785 [==============================] - 308s 392ms/step - loss: 0.9214 - accuracy: 0.7689 - val_loss: 1.0221 - val_accuracy: 0.7347
Epoch 3/30
785/785 [==============================] - 306s 390ms/step - loss: 0.5972 - accuracy: 0.8629 - val_loss: 0.9221 - val_accuracy: 0.7538
Epoch 4/30
785/785 [==============================] - 307s 391ms/step - loss: 0.3741 - accuracy: 0.9273 - val_loss: 0.8921 - val_accuracy: 0.7633
Epoch 5/30
785/785 [==============================] - 306s 390ms/step - loss: 0.2437 - accuracy: 0.9594 - val_loss: 0.8932 - val_accuracy: 0.7769
Epoch 6/30
785/785 [==============================] - 306s 390ms/step - loss: 0.1618 - accuracy: 0.9795 - val_loss: 0.8860 - val_accuracy: 0.7756
Epoch 7/30
785/785 [==============================] - 308s 392ms/step - loss: 0.1083 - accuracy: 0.9907 - val_loss: 0.8982 - val_accura

Save the trained model as `saved_model` format.

In [ ]:
import time
ts = int(time.time())
file_path = f"tf-models/img_classifier/{ts}/"
model.save(filepath=file_path, save_format='tf')

INFO:tensorflow:Assets written to: tf-models/img_classifier/1653575402/assets


In [ ]:
!zip -r /content/tf-models.zip /content/tf-models
from google.colab import files
files.download("/content/tf-models.zip")

  adding: content/tf-models/ (stored 0%)
  adding: content/tf-models/img_classifier/ (stored 0%)
  adding: content/tf-models/img_classifier/1653575402/ (stored 0%)
  adding: content/tf-models/img_classifier/1653575402/variables/ (stored 0%)
  adding: content/tf-models/img_classifier/1653575402/variables/variables.index (deflated 77%)
  adding: content/tf-models/img_classifier/1653575402/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/tf-models/img_classifier/1653575402/saved_model.pb (deflated 91%)
  adding: content/tf-models/img_classifier/1653575402/assets/ (stored 0%)
  adding: content/tf-models/img_classifier/1653575402/keras_metadata.pb (deflated 95%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Load Downloaded Model

In [11]:
!gdown 1H7BSt2k9AVtXsIfipO6KA03WRyNufiDT

Downloading...
From: https://drive.google.com/uc?id=1H7BSt2k9AVtXsIfipO6KA03WRyNufiDT
To: /content/tf-models.zip
100% 26.3M/26.3M [00:00<00:00, 32.5MB/s]


In [14]:
local_zip = './tf-models.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./..')
zip_ref.close()

In [41]:
model = tf.keras.models.load_model('tf-models/img_classifier/1653575402')

# Check its architecture
# model.summary()

## Evaluating Model on Test Data

In [18]:
test_labels = test_datagen.classes

In [19]:
predictions = model.predict(x=test_datagen, verbose=0)

In [20]:
cm = confusion_matrix(y_true=test_labels, 
                      y_pred=predictions.argmax(axis=1))

In [21]:
test_datagen.class_indices

{'aseeda': 0,
 'bakpao': 1,
 'bakso': 2,
 'bbopki': 3,
 'bebek betutu': 4,
 'biryani': 5,
 'bubur ayam': 6,
 'capcay': 7,
 'fu yung hai': 8,
 'gado-gado': 9,
 'gudeg': 10,
 'hotteok': 11,
 'jalebi': 12,
 'jjangmyeon': 13,
 'kebab': 14,
 'kerak telor': 15,
 'kimbab': 16,
 'kimchi': 17,
 'kue keranjang': 18,
 'kulfi': 19,
 'laddu': 20,
 'lumpia': 21,
 'mandu': 22,
 'manggo sticky rice': 23,
 'mie goreng': 24,
 'musakhan': 25,
 'nasi goreng': 26,
 'nasi hainam': 27,
 'oden': 28,
 'onigiri': 29,
 'pempek': 30,
 'pizza': 31,
 'ramen': 32,
 'rawon': 33,
 'rendang': 34,
 'roti maryam': 35,
 'sambosa': 36,
 'samosa': 37,
 'sate': 38,
 'siomay': 39,
 'soto': 40,
 'sushi': 41,
 'tamagoyaki': 42,
 'tempura': 43,
 'tiramisu': 44,
 'tofu': 45,
 'tteokbokki': 46,
 'udon': 47,
 'yakitori': 48}

In [32]:
from sklearn.metrics import accuracy_score, f1_score

In [40]:
F1 = f1_score(y_true=test_labels, y_pred=predictions.argmax(axis=1), average='weighted')
accuracy = accuracy_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))

print(f'Model F1 Score :\n {F1}')
print(f'\nModel Accuracy:\n {accuracy}')

Model F1 Score :
 0.9607486740424126

Model Accuracy:
 0.9612244897959183
